In [1]:
import cv2
import numpy as np

# Load image and convert to HSV color space
img = cv2.imread('basil4l.jpeg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define color ranges for leaves and background
leaf_lower = np.array([35, 50, 50])
leaf_upper = np.array([80, 255, 255])
bg_lower = np.array([0, 0, 0])
bg_upper = np.array([179, 50, 80])

# Threshold the image to get binary masks for leaves and background
leaf_mask = cv2.inRange(hsv, leaf_lower, leaf_upper)
bg_mask = cv2.inRange(hsv, bg_lower, bg_upper)

# Apply morphological opening to remove noise and small objects from the masks
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
leaf_mask = cv2.morphologyEx(leaf_mask, cv2.MORPH_OPEN, kernel)
bg_mask = cv2.morphologyEx(bg_mask, cv2.MORPH_OPEN, kernel)

# Find contours in the binary masks
leaf_contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
bg_contours, _ = cv2.findContours(bg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out any small contours and get the largest contour for leaves and background
leaf_contours = [cnt for cnt in leaf_contours if cv2.contourArea(cnt) > 100]
bg_contours = [cnt for cnt in bg_contours if cv2.contourArea(cnt) > 100]
leaf_contour = max(leaf_contours, key=cv2.contourArea)
bg_contour = max(bg_contours, key=cv2.contourArea)

# Get the bounding rectangle for the leaf contour
x, y, w, h = cv2.boundingRect(leaf_contour)

# Calculate the height of the plant by finding the vertical distance from the base of the stem to the top of the plant
bg_top = min(bg_contour[:, :, 1].flatten())
leaf_top = max(leaf_contour[:, :, 1].flatten())
plant_height = leaf_top - bg_top

# Draw a rectangle around the leaf contour and display the image with the detected leaf count and plant height
cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.putText(img, f"Leaf count: {len(leaf_contours)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#cv2.putText(img, f"Plant height: {plant_height} pixels", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2.imshow('Lemon Basil Plant', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
